In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.optimize as opt
from microlensing import *
import emcee
#import tarfile
import random
import corner

In [2]:
num = 1062 # event number to anaylze, must be in 4-digit format
steps = 5000 # number of steps to walk in MCMC
burn = 600 # number of initial steps to burn in MCMC

In [3]:
urlbeg = "https://kmtnet.kasi.re.kr/~ulens/event/2021/" # beginning of urls used

In [6]:
# dataframe with all necessary columns
eventdata = pd.read_html(urlbeg)[0]
# eventdata[row][column]
# columns: event=0, field=1, t_0=7, t_E=8, u_0=9, Isource=10, Ibase=11

TypeError: argument of type 'method' is not iterable

In [ ]:
# reading pysis.tar.gz; event KMT-2021-BLG-0006, C01_I
#file = pd.read_csv("kmtdata.pysis",sep='\s+',usecols=['HJD','mag','mag_err'])
#df = pd.DataFrame(data=file)

# KMTC42_I.pysis ; event 1062
file = pd.read_csv("KMTC42_I.pysis",usecols=['HJD','mag','mag_err'])
df = pd.DataFrame(data=file)

# KMTC17_I.pysis ; event 1079
#file = pd.read_csv("KMTC17_I.pysis",usecols=['HJD','mag','mag_err'])
#df = pd.DataFrame(data=file)

In [ ]:
# setting up emcee

x = df.HJD
y = df.mag
yerr = df.mag_err

# define prior
def log_prior(theta): 
    t0,log_tE,log_u0,m0,log_fs = theta
    #log_tE = np.log10(tE)
    #log_u0 = np.log10(u0)
    #log_fs = np.log10(fs(m0,ms))
    if 0 < log_tE < 3:
        return -1.0e18
    return 0.0
    if -4. < log_u0 < .5:
        return -1.0e17
    return 0.0
    if -3. < log_fs < np.log10(1.5):
        return -1.0e18
    return 0.0

# define likelihood
def log_likelihood(theta, x, y, yerr):
    t0,log_tE,log_u0,m0,log_fs = theta
    tE = 10**log_tE
    u0 = 10**log_u0
    fs = 10**log_fs
    model = m0 - 2.5*np.log10(1+fs*(a(x,t0,tE,u0)-1))
    sigma_squared = yerr**2
    return -.5 * np.sum((y-model)**2/sigma_squared)

# define probability
def log_probability(theta, x, y, yerr):
    if not np.isfinite(log_prior(theta)):
        return -np.inf
    return log_prior(theta) + log_likelihood(theta, x, y, yerr)

In [ ]:
# initial guesses

def fs(m0,ms):
    return (pow(10,((m0-ms)/2.5)))

t0 = eventlist.t_0[num-1]
tE = eventlist.t_E[num-1]
u0 = eventlist.u_0[num-1]
ms = eventlist.Isource[num-1]
m0 = eventlist.Ibase[num-1]

lininit = [t0,tE,u0,m0,fs(m0,ms)]
loginit = [t0,np.log10(tE),np.log10(u0),m0,np.log10(fs(m0,ms))]

In [ ]:
# define function for model with linear parameters
def m(t,t0,tE,u0,m0,fs):
    return (m0 - 2.5*np.log10(1+fs*(a(t,t0,tE,u0)-1)))

# define function for model that takes log parameters
def logm(t,t0,log_tE,log_u0,m0,log_fs):
    tE = 10**(log_tE)
    u0 = 10**(log_u0)
    fs = 10**(log_fs)
    #return (m0 - 2.5*np.log10(1+fs*(a(x,t0,tE,u0)-1)))
    return m(t,t0,tE,u0,m0,fs)

# fitting model to data using curvefit, best-fit parameters stored in 'popt'
popt,pcov = opt.curve_fit(logm,df.HJD,df.mag,p0=loginit,sigma=df.mag_err)
print(popt)

In [ ]:
# plot data against model with initial linear and log guesses

# plot data:
plt.scatter(df.HJD, df.mag, marker='.', label='data', s=10) #yerr=df.mag_err)
#plt.errorbar(df.HJD, df.mag, yerr=df.mag_err, fmt=".")
plt.xlabel('Time (JD)')
plt.ylim([max(df.mag),min(df.mag)])
plt.ylabel('Magnitude')

# plot models:
#plt.plot(x,m(x,*lininit), label='linear model',color='r')
#plt.plot(x,logm(x,*loginit), label='log model')
plt.plot(x,logm(x,*popt), label='best fit',color='k')

plt.legend()
plt.show()

In [ ]:
# plot log model on its own
#plt.plot(x,logm(x,*loginitial_1062))

In [ ]:
nwalkers = 50
ndim = len(popt)
# number of steps to walk and burn are initialized at top

pos = [popt + 1.0e-3 * np.random.randn(ndim) for i in range(nwalkers)]

sampler = emcee.EnsembleSampler(nwalkers,ndim,log_probability,args=(x,y,yerr))
sampler.run_mcmc(pos,steps,progress=True);
samples = sampler.get_chain(discard=burn, thin=20, flat=True).reshape((-1,ndim))

In [ ]:
#tau = sampler.get_autocorr_time()
#print(tau)

fig, axes = plt.subplots(len(popt), figsize=(10, 7), sharex=True)
samples_plot = sampler.get_chain()
labels = ["t0","log(tE)","log(u0)","m0","log(fs)"]
for i in range(ndim):
    ax = axes[i]
    #for j in range(nwalkers):
    ax.plot(samples_plot[:, :, i], alpha=0.3)
    ax.set_xlim(0, len(samples_plot))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
axes[-1].set_xlabel("step number");
plt.show()
print(popt)

In [ ]:
# creating corner plot of MCMC estimates with percentile labels

figure = corner.corner(
    samples,
    labels=["t0","log(tE)","log(u0)","m0","log(fs)"],
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_kwargs={"fontsize": 12},
);

**prediction part of the code**